# pybind11: binding between Python and C++

1. Wrapping API
2. Python objects, reference counting
3. Python containers

# Why do we use scripting

Computing is about commanding the computers to perform calculations to yeild the results that we want to see.  We want to delegate work to computers as much as possible, when the system performance remains feasible.

In the calculations, scripting plays an important role.  A general architecture of numerical software has the following layers, from high-level to low-level:

* External result
  * This is presented in a non-technical way to people outside the problem-solving team.  They can be stakeholders for business or general public.  The result has to be generated in some way, which may or may not be included in the numerical software we make.
* Problem presentation: physics, math, or equations
  * Users use the software or associated tools to present the technical result.
* Scripting or configuration
  * Users follow the example scripts to configure the problems to solve.  Configuration files may also be used.
* Library interface
  * This defines the application programming interface (API) for the numerical software.  Scripts should not touch anything below this layer.
* Library structure
  * This is where we architect the software.  Good book-keeping code is here to separate the interface and the computing kernel.  Data structures are designed at this layer to make sure no time is wasted in copying or converting data.
* Computing kernel
  * This is the place the does the heavy-lifting, and where we do most of the optimization.

## Research code

For a research code, the boundary between external result, problem presentation, and scripting, and that between library interface, library structure, and computing kernel, may be less clear.  The architecture is usually like:

* Problem presentation: high-level description, physics, and scripting / code configuration
* Library implementation

But sometimes if we don't pay attention to architecting, there may be no boundary between anything.

## Full-fledged application

For a commercial grade package, each of the layers will include more sub-layers.  It is a challenge to prevent those layers or sub-layers from interweaving.  From users' point of view, the sophistication appears in the problem presentation and the scripting layers.  Developers, on the other hand, take care of everything below problem presentation, so that users can focus on problem solving.

## Scripting for modularization

At this point, it should be clear that the scripting layer is the key glue in the system architecture.  The high-level users, who use the code for problem solving, wouldn't want to spend time in the low-level implementation.  Instead, they will specify the performance of the API exposed in the scripting layer.  The performance may be about the quality of result and runtime (including memory).

The scripting layer can separate the programming work between the high-level problem presentation and the low-level library implementation.  A scripting language is usually dynamically typed, while for speed, the low-level implementation language uses static typing system.  In the dynamic scripting language, unit-testing is required for robustness.  In a statically typed language like C++, the compiler and static analyzers are very good at detecting errors before runtime.  But the great job done by the compiler makes it clumsy to use C++ to quickly write highly flexible code for problem presentation.

It is tempting to invent one programming language to rule them all.  That approach needs to convince both the high-level problem solvers and the low-level implementors to give up the tools they are familiar with.  The new language will also need to provide two distinct styles for both use cases.  It will be quite challenging, and before anyone succeeds with the one-language approach, we still need to live with a world of hybrid systems.

# Pybind11

[Pybind11](https://pybind11.readthedocs.io/) is a header-only C++ template library, that allows calling CPython API and provides C++ friendly semantics to allow Python to call C++ constructs and vise versa.

# Build system

Pybind11 is a header-only library, so it doesn't have anything to be built.  When we say 'building' pybind11, we mean to build the project that uses pybind11.

To build pybind11, we need CPython.  It optionally depends on numpy and eigen.  There are several suggested ways to build.  Here list those I think important:

## [Setuptools](https://setuptools.readthedocs.io/en/latest/)

[Setuptools](https://setuptools.readthedocs.io/en/latest/) is an enhancement to Python built-in [distutils](https://docs.python.org/3/library/distutils.html).  Because pybind11 is released to [PyPI](https://pypi.org) as a Python package (https://pypi.org/project/pybind11/), both setuptools and distutils can get the header files and use them to build C++ file that use pybind11.

There is an example for using setuptools to build pybind11 (https://github.com/pybind/python_example/blob/master/setup.py):

```python
from setuptools import setup, Extension
from setuptools.command.build_ext import build_ext
import sys
import setuptools

__version__ = '0.0.1'


class get_pybind_include(object):
    """Helper class to determine the pybind11 include path
    The purpose of this class is to postpone importing pybind11
    until it is actually installed, so that the ``get_include()``
    method can be invoked. """

    def __init__(self, user=False):
        self.user = user

    def __str__(self):
        import pybind11
        return pybind11.get_include(self.user)


ext_modules = [
    Extension(
        'python_example',
        ['src/main.cpp'],
        include_dirs=[
            # Path to pybind11 headers
            get_pybind_include(),
            get_pybind_include(user=True)
        ],
        language='c++'
    ),
]


# As of Python 3.6, CCompiler has a `has_flag` method.
# cf http://bugs.python.org/issue26689
def has_flag(compiler, flagname):
    """Return a boolean indicating whether a flag name is supported on
    the specified compiler.
    """
    import tempfile
    with tempfile.NamedTemporaryFile('w', suffix='.cpp') as f:
        f.write('int main (int argc, char **argv) { return 0; }')
        try:
            compiler.compile([f.name], extra_postargs=[flagname])
        except setuptools.distutils.errors.CompileError:
            return False
    return True


def cpp_flag(compiler):
    """Return the -std=c++[11/14/17] compiler flag.
    The newer version is prefered over c++11 (when it is available).
    """
    flags = ['-std=c++17', '-std=c++14', '-std=c++11']

    for flag in flags:
        if has_flag(compiler, flag): return flag

    raise RuntimeError('Unsupported compiler -- at least C++11 support '
                       'is needed!')


class BuildExt(build_ext):
    """A custom build extension for adding compiler-specific options."""
    c_opts = {
        'msvc': ['/EHsc'],
        'unix': [],
    }
    l_opts = {
        'msvc': [],
        'unix': [],
    }

    if sys.platform == 'darwin':
        darwin_opts = ['-stdlib=libc++', '-mmacosx-version-min=10.7']
        c_opts['unix'] += darwin_opts
        l_opts['unix'] += darwin_opts

    def build_extensions(self):
        ct = self.compiler.compiler_type
        opts = self.c_opts.get(ct, [])
        link_opts = self.l_opts.get(ct, [])
        if ct == 'unix':
            opts.append('-DVERSION_INFO="%s"' % self.distribution.get_version())
            opts.append(cpp_flag(self.compiler))
            if has_flag(self.compiler, '-fvisibility=hidden'):
                opts.append('-fvisibility=hidden')
        elif ct == 'msvc':
            opts.append('/DVERSION_INFO=\\"%s\\"' % self.distribution.get_version())
        for ext in self.extensions:
            ext.extra_compile_args = opts
            ext.extra_link_args = link_opts
        build_ext.build_extensions(self)

setup(
    name='python_example',
    version=__version__,
    author='Sylvain Corlay',
    author_email='sylvain.corlay@gmail.com',
    url='https://github.com/pybind/python_example',
    description='A test project using pybind11',
    long_description='',
    ext_modules=ext_modules,
    install_requires=['pybind11>=2.4'],
    setup_requires=['pybind11>=2.4'],
    cmdclass={'build_ext': BuildExt},
    zip_safe=False,
)```

## Cmake with a sub-directory

When the source tree is put in a sub-directory in your project, as mentioned in the [document](https://pybind11.readthedocs.io/en/stable/compiling.html#building-with-cmake), you can use cmake `add_subdirectory` to include the pybind11:

```cmake
cmake_minimum_required(VERSION 2.8.12)
project(example)

add_subdirectory(pybind11)
pybind11_add_module(example example.cpp)
```

Pybind11 provides the cmake command `pybind11_add_module`.  It set various flags to build your C++ code as an extension module.

## Cmake with installed pybind11

If pybind11 is installed using cmake itself, the `*.cmake` files that pybind11 supplies are installed to the specified location.  It's not needed to write `add_subdirectory` in the `CMakeLists.txt` in your project.

# Wrapping API

Wrapper needs to take care of the differences between the dynamic behaviors in Python and the staticity in C++.  You can directly call pybind11 API.  But a better way is to create another wrapping layer between the pybind11 and your library code.  It allows us to insert additional code in a systematic way.

Here we show a trick for the additional wrapping layer.  See https://github.com/yungyuc/turgon/blob/master/spacetime/include/spacetime/python/WrapBase.hpp:

```cpp
#pragma once

/*
 * Copyright (c) 2017, Yung-Yu Chen <yyc@solvcon.net>
 * BSD 3-Clause License, see COPYING
 */

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <numpy/arrayobject.h>

#include <memory>
#include <type_traits>

PYBIND11_DECLARE_HOLDER_TYPE(T, std::unique_ptr<T>);
PYBIND11_DECLARE_HOLDER_TYPE(T, std::shared_ptr<T>);

#ifdef __GNUG__
#  define SPACETIME_PYTHON_WRAPPER_VISIBILITY __attribute__((visibility("hidden")))
#else
#  define SPACETIME_PYTHON_WRAPPER_VISIBILITY
#endif

namespace spacetime
{

namespace python
{

/**
 * Helper template for pybind11 class wrappers.
 */
template< class Wrapper, class Wrapped, class Holder = std::unique_ptr<Wrapped>, class WrappedBase = Wrapped >
class
SPACETIME_PYTHON_WRAPPER_VISIBILITY
WrapBase {

public:

    using wrapper_type = Wrapper;
    using wrapped_type = Wrapped;
    using wrapped_base_type = WrappedBase;
    using holder_type = Holder;
    using base_type = WrapBase< wrapper_type, wrapped_type, holder_type, wrapped_base_type >;
    using class_ = typename std::conditional<
        std::is_same< Wrapped, WrappedBase >::value
      , pybind11::class_< wrapped_type, holder_type >
      , pybind11::class_< wrapped_type, wrapped_base_type, holder_type >
    >::type;

    static wrapper_type & commit(pybind11::module * mod, const char * pyname, const char * clsdoc) {
        static wrapper_type derived(mod, pyname, clsdoc);
        return derived;
    }

    WrapBase() = delete;
    WrapBase(WrapBase const & ) = default;
    WrapBase(WrapBase       &&) = delete;
    WrapBase & operator=(WrapBase const & ) = default;
    WrapBase & operator=(WrapBase       &&) = delete;
    ~WrapBase() = default;

#define DECL_ST_PYBIND_CLASS_METHOD(METHOD) \
    template< class... Args > \
    /* NOLINTNEXTLINE(bugprone-macro-parentheses) */ \
    wrapper_type & METHOD(Args&&... args) { \
        m_cls.METHOD(std::forward<Args>(args)...); \
        return *static_cast<wrapper_type*>(this); \
    }

    DECL_ST_PYBIND_CLASS_METHOD(def)
    DECL_ST_PYBIND_CLASS_METHOD(def_readwrite)
    DECL_ST_PYBIND_CLASS_METHOD(def_property)
    DECL_ST_PYBIND_CLASS_METHOD(def_property_readonly)
    DECL_ST_PYBIND_CLASS_METHOD(def_property_readonly_static)

#undef DECL_ST_PYBIND_CLASS_METHOD

protected:

    WrapBase(pybind11::module * mod, const char * pyname, const char * clsdoc)
      : m_cls(*mod, pyname, clsdoc)
    {}

private:

    class_ m_cls;

}; /* end class WrapBase */

} /* end namespace python */

} /* end namespace spacetime */

/* vim: set et ts=4 sw=4: */
```

Use `WrapGrid`, `WrapField`, `WrapSolver`, `WrapCelm`, and `WrapSelm` to show how `WrapBase` is used.  See https://github.com/yungyuc/turgon/blob/master/spacetime/include/spacetime/python/wrapper_spacetime.hpp:

```c++
#pragma once

/*
 * Copyright (c) 2018, Yung-Yu Chen <yyc@solvcon.net>
 * BSD 3-Clause License, see COPYING
 */

#include "spacetime/python/common.hpp"

namespace spacetime
{

namespace python
{

class
SPACETIME_PYTHON_WRAPPER_VISIBILITY
WrapGrid
  : public WrapBase< WrapGrid, Grid, std::shared_ptr<Grid> >
{

    friend base_type;

    WrapGrid(pybind11::module * mod, const char * pyname, const char * clsdoc)
      : base_type(mod, pyname, clsdoc)
    {
        namespace py = pybind11;
        (*this)
            .def(
                py::init([](real_type xmin, real_type xmax, size_t nelm) {
                    return Grid::construct(xmin, xmax, nelm);
                }),
                py::arg("xmin"), py::arg("xmax"), py::arg("nelm")
            )
            .def(
                py::init([](xt::pyarray<wrapped_type::value_type> & xloc) {
                    return Grid::construct(xloc);
                }),
                py::arg("xloc")
            )
            .def("__str__", &detail::to_str<wrapped_type>)
            .def_property_readonly("xmin", &wrapped_type::xmin)
            .def_property_readonly("xmax", &wrapped_type::xmax)
            .def_property_readonly("ncelm", &wrapped_type::ncelm)
            .def_property_readonly("nselm", &wrapped_type::nselm)
            .def_property_readonly(
                "xcoord",
                static_cast<wrapped_type::array_type & (wrapped_type::*)()>(&wrapped_type::xcoord)
            )
            .def_property_readonly_static("BOUND_COUNT", [](py::object const &){ return Grid::BOUND_COUNT; })
        ;
    }

}; /* end class WrapGrid */

class
SPACETIME_PYTHON_WRAPPER_VISIBILITY
WrapField
  : public WrapBase< WrapField, Field, std::shared_ptr<Field> >
{

    friend base_type;

    WrapField(pybind11::module * mod, const char * pyname, const char * clsdoc)
      : base_type(mod, pyname, clsdoc)
    {
        namespace py = pybind11;
        (*this)
            .def("__str__", &detail::to_str<wrapped_type>)
            .def_property_readonly("grid", [](wrapped_type & self){ return self.grid().shared_from_this(); })
            .def_property_readonly("nvar", &wrapped_type::nvar)
            .def_property(
                "time_increment",
                &wrapped_type::time_increment,
                &wrapped_type::set_time_increment
             )
            .def_property_readonly("dt", &wrapped_type::dt)
            .def_property_readonly("hdt", &wrapped_type::hdt)
            .def_property_readonly("qdt", &wrapped_type::qdt)
            .def(
                "celm",
                static_cast<Celm (wrapped_type::*)(sindex_type, bool)>(&wrapped_type::celm_at<Celm>),
                py::arg("ielm"), py::arg("odd_plane")=false
            )
            .def(
                "selm",
                static_cast<Selm (wrapped_type::*)(sindex_type, bool)>(&wrapped_type::selm_at<Selm>),
                py::arg("ielm"), py::arg("odd_plane")=false
            )
        ;
    }

}; /* end class WrapField */

class
SPACETIME_PYTHON_WRAPPER_VISIBILITY
WrapSolver
  : public WrapSolverBase< WrapSolver, Solver >
{

    using base_type = WrapSolverBase< WrapSolver, Solver >;
    using wrapper_type = typename base_type::wrapper_type;
    using wrapped_type = typename base_type::wrapped_type;

    friend base_type;
    friend base_type::base_type;

    WrapSolver(pybind11::module * mod, const char * pyname, const char * clsdoc)
      : base_type(mod, pyname, clsdoc)
    {
        namespace py = pybind11;
        (*this)
            .def
            (
                py::init(static_cast<std::shared_ptr<wrapped_type> (*) (
                    std::shared_ptr<Grid> const &, typename wrapped_type::value_type, size_t
                )>(&wrapped_type::construct))
              , py::arg("grid"), py::arg("time_increment"), py::arg("nvar")
            )
        ;
    }

}; /* end class WrapSolver */

class
SPACETIME_PYTHON_WRAPPER_VISIBILITY
WrapCelm
  : public WrapCelmBase< WrapCelm, Celm >
{

    using base_type = WrapCelmBase< WrapCelm, Celm >;
    friend base_type::base_type::base_type;

    WrapCelm(pybind11::module * mod, const char * pyname, const char * clsdoc)
      : base_type(mod, pyname, clsdoc)
    {}

}; /* end class WrapCelm */

class
SPACETIME_PYTHON_WRAPPER_VISIBILITY
WrapSelm
  : public WrapSelmBase< WrapSelm, Selm >
{

    using base_type = WrapSelmBase< WrapSelm, Selm >;
    friend base_type::base_type::base_type;

    WrapSelm(pybind11::module * mod, const char * pyname, const char * clsdoc)
      : base_type(mod, pyname, clsdoc)
    {}

}; /* end class WrapSelm */

} /* end namespace python */

} /* end namespace spacetime */

// vim: set et sw=4 ts=4:
```

## Function and properties

## Named and keyword arguments

# Python objects and reference counting

# Python containers

# Exercises

# References

1. S.C. Chang, "The Method of Space-Time Conservation Element and Solution Element -- A New Approach for Solving the Navier-Stokes and Euler Equations," J. Comput. Phys., 119, pp. 295-324, (1995).  DOI: [10.1006/jcph.1995.1137](https://doi.org/10.1006/jcph.1995.1137)